<a href="https://colab.research.google.com/github/Lucy-Moctezuma/SFSU-CodeLab-Work-/blob/main/E.%20Coli%20Machine%20Learning%20Project/2_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Logistic Regression**

## ***Objectives for this Notebook***
- Learn the basics of how logistic regression model works.
- Create functions to implement Logistic Regression to Moradigaravand's dataset.

**Logistic Regression** is a classification model that allows us to predict the probability for a binary outcome (2 classes). Typically it is expected that the threshold for logistic regression is 0.5; In our example, this means that above this probability, the model would predict R(Resistant) and below this it will predict S(Susceptible).

The equation for Logistic Regression is actually derived from Linear Regression, but instead of Y (Response) we have the natural log of odds:

$$ ln(\frac{P}{1-P}) = \hat \beta_0 + \hat \beta_jX$$

After isolating P, we end up with the equation below:

$$P = \frac{e^{\hat{\beta}_{0}+\hat{\beta}_{j}X}}{1+e^{\hat{\beta}_{0}+\hat{\beta}_{j}X}}$$

- ***P*** is the probability of an outcome. Therefore P is a number between 0(0%) and 1(100%). Our threshold would be 50%, for our example if P < 0.5 our model would predict Susceptibility (S) and if P > 0.5 our model would predict Resistance (R)

- $\hat \beta_0$ is the intercept term and $\hat \beta_j = [\beta_1 , \beta_2 , \beta_3, ... , \beta_{18293}]$ are the coefficients for each of our features, which the model will try to estimate using our data, there is one coefficient per column feature we are using making 18293 of them in our example.

- $X = [Year \ columns + Gene \ Absence \ and \ Presence \ Columns + Population \ Structure \ Columns]$

We will see each of the parts of this equation as we go along in the tutorial, so we can have better picture of these.

A full understanding of the math behind the logistic regression model is not necessary. Using a logistic regression model (or any other machine learning model) doesn't require a detailed understanding of the math behind it.




### **1) Importing Packages needed**


In [ ]:
# Data manipulation imports for ML
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Import packages for logistic regression model
from sklearn.linear_model import LogisticRegression
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

# Imports for model evaluation
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Imports for data visualization
import matplotlib.pyplot as plt

# Imports for file management
import os
from google.colab import drive
drive.mount('/content/drive')

### **2) Loading CSV file and creating dataframes for each antibiotic**
In here we will be Loading the CSV we created in the previous notebook. This file should contain all out antibiotic drugs (Labels) and all the Years of isolation , Gene Presence/Absence data and the Population Structure data (Features).


In [ ]:
# Loads csv file as a dataframe
filepath = '/content/drive/My Drive/EColi_ML_CSV_files/'

# reads csv file as a dataframe
All_Drugs_df = pd.read_csv(filepath+"EColi_Merged_dfs.csv", na_values="NaN")
All_Drugs_df.head()

### **3) Separating each Drug Dataframe into 4 sections : Training features, training labels, testing features and testing labels.**

The objective of this part will be to create a single dataframe for each antibiotic drug. This means that we want a dataframe with all our features and the label for only one drug. This is because Resistance and Susceptibility are not universal. For example, just because an isolate of *E. coli* is resistant to say AMP (Ampicilin), it doesn't mean that is resistant to AMX (Amoxicilin).

Below we can check the list of antibiotics again:




In [ ]:
#here we make a list of the columns we want to keep: the column for the isolate, the column for the drug we are interested in and all features (starting from column 13).
drug_list = All_Drugs_df.iloc[:,1:13].columns
drug_list

After this we will need to split each of our 12 antibiotic dataframes into 4 different sections:

**TRAINING**

**a) labels_train:** are the labels (S or R) for a single antibiotic drug that will be used to teach our model how to make predictions.

**b) features_train:** are the features that will be used along with the labels_train to teach our model to make predictions. Note that this is actually the X matrix in our logistic equation! They will be used to estimate our $\beta_0$ and all the $\beta_j$ values with a process called *Maximum Likelihood*. You can watch the mathematical details of how this is done by watching this [video](https://www.youtube.com/watch?v=BfKanl1aSG0) by Josh Starmer

**TESTING**

**c) labels_test:** are the labels we will holding out so that we can see at the end if we made accurate predictions.

**d) features_test:** are the X values we will plug into our model, once $\beta_0$ and all the $\beta_j$ values have already been estimated.

- Below we create a function that will be used to separate each of our 12 dataframes into the 4 separate parts described above. Specifically we also specified that 33% of our data to be used as a testing set and thus 67% of our data remains to become our training set. You can choose a different percentage to split them, but know that the majority of our data should be used for training, other splits people try are 30/70 or 20/80 for testing/training respectively.

- In addition, the function will save each of these 4 parts into a python Dictionary object. If you are unfamiliar with what a dictionary is in python, feel free to check out this useful [link](https://docs.python.org/3/tutorial/datastructures.html#dictionaries). This way we can organize and access our 4 data chunks for a specific antibiotic drug.

**The function created below will create a dataframe and split the data in Training (labels and features) and Testing (labels and features) for each antibiotic.**

In [ ]:
# Separating each dataframe into Labels and Features for training and testing data.
# Our function uses the handy train_test_split() function.

def Split_train_test(drug):
  #here we make a list of the columns we want to keep: the column for the isolate, the column for the drug we are interested in and all features (starting from column 13).
  df_list = [All_Drugs_df[["Isolate",drug]], All_Drugs_df.iloc[:,13:]]

  #here we create a data frame with just the columns we wanted to keep.
  Drug_df = pd.concat(df_list, axis=1)

  #here we drop all rows with missing data
  Drug_df = Drug_df.dropna()

  # Creating a dictionary to store each antibiotic datasets
  Train_test_dic = {}

  # Defining the label columns
  labels = Drug_df[drug]

  # Defining features columns
  features = Drug_df.drop(columns=[drug])

  # Separating training (features and labels) and testing (features and labels) datasets
  features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.33, random_state=42, stratify=labels)

  # storing each data chunk in a dictionary
  Train_test_dic['labels_train'] = labels_train
  Train_test_dic['features_train'] = features_train
  Train_test_dic['labels_test'] = labels_test
  Train_test_dic['features_test'] = features_test

  return Train_test_dic

In the code below we can see what our CTZ dictionary contains the 4 chunks explained at the beggining of part 3. We have 1296 training observations and 639 observations were reserved as testing.


In [ ]:
# Implementing the function Split_train_test() for CTZ example
CTZ_Train_test_dic = Split_train_test("CTZ")

# checking the shape of each dataframe or series stored in the dictionary created for drug CTZ
print("CTZ")
for k, df in CTZ_Train_test_dic.items():
  print(k, df.shape)

Below we can see a count of Susceptible and Resistance strains for the Training and Testing datasets. Notice that there are many more Susceptible than Resistant *E. coli* to CTZ drug. This is considered an "inbalanced" dataset. Later on we will discuss how certain metrics may not be as reliable as others when looking at an imbalanced dataset.

In [ ]:
# Accessing a particular chunk of data
print("Class Counts for Training Dataset:")
CTZ_Train_test_dic["labels_train"].value_counts()

In [ ]:
# Accessing a particular chunk of data
print("Class Counts for Testing Dataset:")
CTZ_Train_test_dic["labels_test"].value_counts()

### **4) Creating different combination of features before training**

The next part of this project is to add some complexity in our analysis by choosing specifically what sort of features we would like to include. In the original Moradigaravand paper, the authors did the same thing, so we follow their example.
Below we have chosen to a specific list of combination of features we are interested in.

Recall that we have 3 types of different features we went on detail on our [first notebook](https://colab.research.google.com/drive/1dHGfZGGrSUliH14iVKX-r7pAl_IGiMqm?usp=sharing):
- **G**: Gene presence and absence
- **S**: Population Structure
- **Y**: Years of Isolation

We are interested in the following combinations: **G, S, GY, SY, GS, GYS.** This means that for any possible combination of feautures, we will train and test the machine learning models. Then later we can determine which combination of features gives the best results.

Below we create a function that will take the features dataframe (train or test) from the dictionary we have created in part 3 and then will create different feature combinations dataframe. For instance, a **"GS"** combination implies that we would like to use the Gene presence and absence (G) feature columns and the Years of isolation (Y) columns.

In [ ]:
# making a list of combinations of data sources we would like to test in our ML models
combo_list = ['G', 'S', 'GY', 'SY', 'GS', 'GYS']

# making a function that creates different feature combinations of the predictor features
def combo_feat(features_df, drug, combo):

  # creating Year column filters for features_df
  year_filter = [col for col in features_df if col.startswith("Year")]
  year_feat = features_df[year_filter]

  # creating Population structure column filters for features_df
  pop_str_filter = [col for col in features_df if col.startswith("cutoff")]
  pop_struc_feat = features_df[pop_str_filter]

  # creating Gene precence column filters for features_df
  gene_presc_filter = [col for col in features_df.columns if col not in pop_str_filter and col not in year_filter and col != "Isolate"]
  gene_presc_feat = features_df[gene_presc_filter]

  if combo == 'G':
    df_list = [features_df['Isolate'], gene_presc_feat]
    G_feat_df = pd.concat(df_list, axis=1)
    G_feat_df = G_feat_df.drop(columns=['Isolate'])
    return G_feat_df

  if combo == 'S':
    df_list = [features_df['Isolate'], pop_struc_feat]
    S_feat_df = pd.concat(df_list, axis=1)
    S_feat_df = S_feat_df.drop(columns=['Isolate'])
    return S_feat_df

  if combo == 'GY':
    df_list = [features_df['Isolate'], gene_presc_feat, year_feat]
    GY_feat_df = pd.concat(df_list, axis=1)
    GY_feat_df = GY_feat_df.drop(columns=['Isolate'])
    return GY_feat_df

  if combo == 'SY':
    df_list = [features_df['Isolate'], pop_struc_feat, year_feat]
    SY_feat_df = pd.concat(df_list, axis=1)
    SY_feat_df = SY_feat_df.drop(columns=['Isolate'])
    return SY_feat_df

  if combo == 'GS':
    df_list = [features_df['Isolate'], gene_presc_feat, pop_struc_feat]
    GS_feat_df = pd.concat(df_list, axis=1)
    GS_feat_df = GS_feat_df.drop(columns=['Isolate'])
    return GS_feat_df

  if combo == 'GYS':
    df_list = [features_df['Isolate'], gene_presc_feat, year_feat, pop_struc_feat, ]
    GYS_feat_df = pd.concat(df_list, axis=1)
    GYS_feat_df = GYS_feat_df.drop(columns=['Isolate'])
    return GYS_feat_df

In [ ]:
# Implementing combo_feat() function created for training data
CTZ_GYS_train_df = combo_feat(CTZ_Train_test_dic['features_train'],"CTZ","GYS")

# looking only at the feature column names for the combination for "GYS" for drug "CTZ" for training data
CTZ_GYS_train_df.columns

### **5) Creating Logistic regression model and training it per feature combination**

The next step involves creating a function that will actually creat our Logistic Regression model and train it on our desired combination of training features and the labels for the drug we choose. While this function seems fairly straight forward, there is a lot of calculations happening in the backrgound when we call the LG.fit() function which trains our model. We will only take a small peak at what's going on in the background. Running this function will also take more time than the other functions we have run until now.


In [ ]:
# creating Logistic regression model function
@ignore_warnings(category=ConvergenceWarning)
def run_LG(feat_train_df, lab_train, drug, combo):
  print(drug +" Training combo: "+ combo)
  LG = LogisticRegression(random_state = 42, solver= 'lbfgs', C=1.0, max_iter=500, class_weight='balanced')
  LG = LG.fit(feat_train_df, lab_train)
  return LG

In [ ]:
# implementing run_LG() for specific drug feature combination dataframe
LG_CTZ_GYS_model = run_LG(CTZ_GYS_train_df, CTZ_Train_test_dic['labels_train'],"CTZ","GYS")
LG_CTZ_GYS_model

Below we can see that our model contains several coefficients. We will printout first the intercept ($\hat{\beta}_{0}$), then a list of the coefficients that correspond to each column feature ($\hat{\beta}_{j}$) and finally we can get a print out of the total number of coefficients in our model, these should be the same number as all the features we used in this example.

In [ ]:
# printing the beta_0 or intercept value of our model
print("Intercept:",LG_CTZ_GYS_model.intercept_[0])

# printing all the beta_j's or coefficients of our logistic regression model
print("All beta_j values:", LG_CTZ_GYS_model.coef_[0])

# printing the number of all the beta_j values
print("Number of beta_j values: ", len(LG_CTZ_GYS_model.coef_[0]))

### **6) Making predictions from Logistic regression model**

Now that our model has been trained and all $\beta$ values have been estimated, we are ready to make predictions! We will use the features of our testing data which we separated when we made our antibiotic drug dictionary.

Below we create another function where we predict labels using the actual model and the "features_test" chunk.


In [ ]:
# creating a function using the model created and trained and the feature combinations from testing data
def predict(LG_combo_Model, features_test):
  labels_pred = LG_combo_Model.predict(features_test)
  return labels_pred

Below we will use the function **combo_feat()** to split the testing dataset

In [ ]:
# Implementing combo_feat() function created for testing data
CTZ_GYS_test_df = combo_feat(CTZ_Train_test_dic['features_test'],"CTZ","GYS")

The we will implement the function **predict()** below:

In [ ]:
# Implementation of the predict() function using the feature combination "GYS"
CTZ_GYS_labels_pred = predict(LG_CTZ_GYS_model,CTZ_GYS_test_df)

# observe how many predictions were made for each category "R" and "S"
print("Labels predicted: ", np.unique(CTZ_GYS_labels_pred, return_counts=True))


We can see in the last output that index order for **R (Resistance is 0)** and for **S (Susceptible is 1)**, meaning that we predicted 120 Resistant and 519 Susceptible samples.

In [ ]:
# let's look at the predictions for the first 30 isolates
print("Labels predicted for first 10 test isolates: ", CTZ_GYS_labels_pred[:30])

# and the actual labels for the first 230 isolates – do they match?
print("Labels predicted for first 10 test isolates: ", np.array(CTZ_Train_test_dic['labels_test'][:30]))


### **7) Evaluating our model using a confusion matrix and metrics**
There are different ways we can evaluate our model. A **confusion matrix** is a plot showing a prediction summary for our model. It allows us to see how many predictions were correct and incorrect. There are also different metrics we can calculate using this graph. For this tutorial we will focus on three of them: **Accuracy**, **Recall** and **Precision**.

- **Accuracy:** is the total number of correct classifications over the total amount of predictions made.

- **Recall:** is the number of correct classifications made for a particular class over all predictions of that class.

- **Presicion:** is the number of classifications made for a particular class over the actual number of strains for that class.

Recall and Precision can each be calculated for resistance and for susceptibility.

When we have two classes, a 2 by 2 confusion matrix contains:

- **True Positives (TP)** = Resistant strains correctly classified as resistant (R) = 48
- **True Negatives (TN)** = Susceptible strains correctly classified as susceptible (S) = 479
- **False Positives (FP)** = Susceptoble strains incorrectly classified as resistant (R) = 72
- **False Negatives (FN)** = Resistant strains incorrectly classified as susceptible (S) = 40

In [ ]:
# Creating a function that evaluates our model using our actual and predicted data
def evaluate(LG_combo_model, labels_test, labels_pred, cf= True, show_results=True):
  report = classification_report(labels_test, labels_pred, output_dict = True)
  if cf == True:
    cm = confusion_matrix(labels_test, labels_pred, labels=LG_combo_model.classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=LG_combo_model.classes_)
    disp.plot()
    plt.show()
  if show_results == True:
    print("Results")
    print('Accuracy:',report['accuracy'])
    print('R recall:',report['R']['recall'])
    print('S recall:',report['S']['recall'])
    print('R precision:',report['R']['precision'])
    print('S precision:',report['S']['precision'])
  return [report['accuracy'], report['R']['recall'], report['S']['recall'], report['R']['precision'], report['S']['precision']]

Before we implement our function we can show a manual way in which these metrics are calculated, first for the overall *Accuracy* and then *Recall* and *Precision* for only the Resistant strains (R) as an example:

|<font size=3>Metrics|<font size=3>General formula| <font size=3>Formula for 2 classes|<font size=3>Manual Calculation|
|--|:-:|:-:|:--|
|<font size=3>**Accuracy**|<font size=3>$\frac{Correctly \ classified}{All \ Predicted}$|<font size=3>$\frac{TP + TN}{TP + TN + FN + FP}$|<font size=3>$\frac{48 + 479}{639} = 0.8247$|
|<font size=3>**R Recall:**|<font size=3>$\frac{Correctly \ classified \ as \ R}{All \ Actual \ R}$|<font size=3>$\frac{TP}{TP + FN}$|$\frac{48}{48 + 40} = 0.545$|
|<font size=3>**R Precision:**|<font size=3>$\frac{Correctly \ classified \ as \ R}{All \ Predicted \ R}$|<font size=3>$\frac{TP}{TP + FP}$|<font size=3>$\frac{48}{48 + 72} = 0.4$|


**NOTE:** In this tutorial we only work with 2 classes (R and S), thus the abbreviations (TP, TN, FP and FN) apply to our confusion matrix, however for situations with more than 2 classes, refer to the general formula column.

In [ ]:
# implementing the evaluate() function
Model_Report = evaluate(LG_CTZ_GYS_model, CTZ_Train_test_dic['labels_test'],CTZ_GYS_labels_pred)

Note that in this example, accuracy is 82%. That sounds quite good. But nearly half of the resistant cases are misclassified as susceptible. So, if the main goal here is to detect resistant strains, this particular model does not perform well at all.

### **8) Use all functions and evaluate every drug in every feature combination!**


#### **a) Lets recall the list of drugs we have available and the combination of features we are interested in**

In [ ]:
# let's check all drugs
print(drug_list)

# let's see all feature combinations we are interested in
print(combo_list)

#### **b) Create a loop that will go through all our functions using the lists above**

The code below will take longer to run because we will train our logistic regression using all feature combinations for each of the antibiotic drugs. it will then be stored in a python dictionary.

**NOTE:** Python Dictionaries, are common data structures within the python language that are very efficient at storing and retrieving information. It consists in key-value pairs, in our case each key is: drug_combo and the value is paired with are the metrics results.

In [ ]:
# Lets use all our functions this time and save our report into a single data structure
LG_model_metrics = {}

for drug in drug_list:
  print(drug)
  # splits each drug df into a dictionary with testing and training data
  Test_Train_dic = Split_train_test(drug)
  for combo in combo_list:
    # Training each drug and combo features
    labels_train = Test_Train_dic["labels_train"]
    # create corresponding feature_df for training
    features_train = combo_feat(Test_Train_dic["features_train"], drug, combo)
    # runs logistic regression model using the corresponding training feature_df
    LG_combo_model = run_LG(features_train, labels_train, drug, combo)
    # create corresponding feature_df for testing
    features_test = combo_feat(Test_Train_dic["features_test"], drug, combo)
    # generate predictions based on the feature combination tested
    labels_pred = predict(LG_combo_model, features_test)
    # loading the actual labels
    labels_test = Test_Train_dic["labels_test"]
    # Evaluating our model
    report = evaluate(LG_combo_model, labels_test, labels_pred, cf=False, show_results=False)
    # Saving the results into a dictionary
    LG_model_metrics[drug+"_"+combo] = report
    print(report)

#### **b) Store the metrics report for all drugs and features combinations as a csv file**

Using the dictionary we created above to create a final dataframe for all our results and store them as a csv file we can access afterwards.

In [ ]:
# convert dictionary into a dataframe
LG_metrics = pd.DataFrame.from_dict(LG_model_metrics, orient='index',columns=["Accuracy", "R_recall", "S_recall", "R_precision", "S_precision"]).reset_index()
LG_metrics = LG_metrics.rename(columns = {'index':'Drug_combo'})

# saving our metric results into a CSV file
LG_metrics.to_csv(filepath+"LG_metrics_df.csv", index= False)
LG_metrics

#### **c) Create a bar graph showing metrics for all drugs when using all three types of features (GYS)**
Below we will create a bargraph to visualize metrics for every antibiotic that uses the feature combination GYS (Gene Presence and Absence, Year of Isolate collection and Population Structure). In the graph below we are comparing Accuracy and Recall scores for both classes. Notice for instance the antibiotic **CTZ**, when we only pay attention to the Accuracy, we would conclude that our model is doing fairly well, however, when examine recall we can clearly see that the Recall for Resistance is noticeable lower than for Susceptibility. Therefore, for imbalanced classes, Accuracy may not always be the best metric to use, or at least not without also looking at other metrics.

The code below serves to create a new directory to store all our Machine Learning plots and figures created. If you run the code more than once it will give an error, that you can ignore this message, since the directory has already been created.

In [ ]:
# makes a directory for all your plot images
os.mkdir('/content/drive/My Drive/EColi_ML_Plots')

In [ ]:
# filtering for all the rows that contain GYS features
GYS_filter = [drug_combo for drug_combo in LG_metrics['Drug_combo'] if drug_combo.endswith("GYS")]
GYS_df = LG_metrics.loc[LG_metrics["Drug_combo"].isin(GYS_filter)]

# Figure Size
fig = plt.figure(figsize =(20, 8))

# Adding title
plt.title('Accuracy, R_recall, S_recall', fontsize = 12)

# Variables to be plotted
x = np.arange(len(GYS_df["Drug_combo"]))
acc = list(GYS_df["Accuracy"])
R_rec = list(GYS_df["R_recall"])
S_rec = list(GYS_df["S_recall"])

# Plotting barcharts
acc_bar=plt.bar(x-0.25, height= acc, width=0.25, color="grey", edgecolor="gray")
rrec_bar=plt.bar(x, height= R_rec, width=0.25, color="plum", edgecolor="gray")
srec_bar=plt.bar(x+0.25, height= S_rec, width=0.25, color="lavenderblush", edgecolor="gray")

plt.xticks([r for r in range(len(GYS_df["Drug_combo"]))],
        GYS_df["Drug_combo"], fontsize = 12)

#legend
fig.legend([acc_bar,rrec_bar,srec_bar],["Accuracy", "R_recall", "S_recall"], bbox_to_anchor=(0.4,-0.35, 0.04, 0.4), fontsize=12)

# Saving bargraph into our new directory
plt.savefig('/content/drive/My Drive/EColi_ML_Plots/GYS_LG_Accuracy_and_Recall_Scores.jpg',dpi=400, bbox_inches="tight")

# Show Plot
plt.show()

Thanks for making it this far! We have accomplished the objectives of this notebook. We learned the basics of how logistic regression works and learned how to make functions in order to run this model using Moragadivand's *E. coli* dataset. Now we will move on to our next ML model, [Random Forest](https://colab.research.google.com/drive/13XuIu7AsR6HTEerT9lyoLn3lWQjUvXg-?usp=sharing) and learn a bit more about tree-based models in general.